## Cleaning

### Text Cleaning

#### 1. text cleaning.py

In [ ]:
"""
gemini 1.5 flash LLM을 이용하여, filtering 결과로 생성된 text_noise.csv의 text를 정제하는 코드입니다.
정제 결과는 text_cleaned_raw.csv에 저장되며, postprocess.py를 통해 후처리해야 합니다.
"""

INPUT_FILE = "../1_filtering/5_text_noise.csv"
OUTPUT_FILE = "text_cleaned_raw.csv"

import pandas as pd
import os
import time
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# API 키 설정
os.environ["GOOGLE_API_KEY"] = " "

# 데이터 로드
data = pd.read_csv(INPUT_FILE)

# 모델 초기화
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")

# 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    input_variables=["noisy_text"],
    template="""
    다음은 노이즈가 포함된 텍스트입니다:
    "{noisy_text}"
    이 텍스트에서 의미를 유지하면서 노이즈를 제거하고, 자연스럽고 뉴스 제목처럼 간결하게 변환해줘.
    다른 문구 말고 오직 변환된 텍스트만을 출력해줘.
    """
)

# 결과를 저장할 리스트
results = []

# 배치 크기 설정
batch_size = 10
total_rows = len(data)

# 기존 결과가 있으면 불러오기
if os.path.exists(OUTPUT_FILE):
    existing_results = pd.read_csv(OUTPUT_FILE)
    processed_ids = set(existing_results['ID'])
    results = existing_results.to_dict('records')
    print(f"기존에 {len(processed_ids)}개의 데이터를 처리했습니다. 이어서 진행합니다.")
else:
    processed_ids = set()
    print("처음부터 시작합니다.")

# 배치 처리
for start_idx in range(0, total_rows, batch_size):
    end_idx = min(start_idx + batch_size, total_rows)
    batch = data.iloc[start_idx:end_idx]

    for index, row in batch.iterrows():
        id = row['ID']
        if id in processed_ids:
            continue  # 이미 처리된 데이터는 건너뜁니다

        noisy_text = row['text']
        target = row['target']

        # 프롬프트 생성
        prompt = prompt_template.format(noisy_text=noisy_text)

        success = False
        retry_count = 0
        max_retries = 5

        while not success and retry_count < max_retries:
            try:
                # 모델 예측
                response = model.invoke(prompt)

                # AIMessage 객체에서 실제 텍스트 추출
                cleaned_text = response.content.strip()

                # 결과 저장
                results.append({
                    "ID": id,
                    "Original Text": noisy_text,
                    "Cleaned Text": cleaned_text,
                    "target": target
                })

                processed_ids.add(id)
                print(f"ID {id} 처리 완료.")
                success = True

                # 요청 사이에 지연 시간 추가
                time.sleep(2)  # 지연 시간을 늘려서 API 부하를 줄입니다

            except Exception as e:
                retry_count += 1
                print(f"ID {id} 처리 중 오류 발생: {e}")
                if 'ResourceExhausted' in str(e):
                    wait_time = 30  # 쿼터 초과 시 1분 대기
                    print(f"{wait_time}초 후에 재시도합니다.")
                    time.sleep(wait_time)
                else:
                    print(f"10초 후에 재시도합니다.")
                    time.sleep(10)

        if not success:
            print(f"ID {id} 처리를 건너뜁니다.")
            # 실패한 경우에도 결과에 추가 (빈 문자열로)
            results.append({
                "ID": id,
                "Original Text": noisy_text,
                "Cleaned Text": "",
                "target": target
            })
            processed_ids.add(id)

    # 중간 저장
    results_df = pd.DataFrame(results)
    results_df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')
    print(f"{end_idx}/{total_rows}개 데이터 처리 완료. 중간 결과를 저장했습니다.")

print(f"모든 데이터 처리가 완료되었습니다. 결과는 '{OUTPUT_FILE}'에 저장되었습니다.")


#### 2. postprocess_text_cleaning.py

In [ ]:
"""
text_cleaning.py를 통해 생성한 text_cleaned_raw.csv를 후처리하는 코드입니다.
후처리 결과는 text_cleaned.csv, text_cleaned_nida.csv로 저장됩니다.
"""

INPUT_FILE = "text_cleaned_raw.csv"
OUTPUT_FILE = "text_cleaned.csv"
OUTPUT_NIDA_FILE = "text_cleaned_nida.csv"

import pandas as pd
import re

df = pd.read_csv(INPUT_FILE)

# 1. 컬럼 제거
df.drop(columns=['Original Text'], inplace=True) # Original Text 컬럼 제거
df.dropna(subset=['Cleaned Text'], inplace=True) # Cleaned Text 컬럼 결측치 제거


# 2. 응답 오류 처리
##  '->'이 포함된 문장에서 '->' 앞 문장 제거 ('문장 A -> 문장 B' ~> '문장 B')
df['Cleaned Text'] = df['Cleaned Text'].apply(
    lambda x: x.split('->', 1)[1].strip() if '->' in x else x
)

## '뉴스 제목:'이 포함된 문장에서 '뉴스 제목:' 앞 문장 제거
df['Cleaned Text'] = df['Cleaned Text'].str.extract(
    r'뉴스 제목:\s*(.*)', expand=False
).fillna(df['Cleaned Text'])

## 답변에 "니다."를 포함하는 행을 분리해 nida.csv에 저장
contains_nida = df[df['Cleaned Text'].str.contains("니다.", regex=False)]
df = df[~df['Cleaned Text'].str.contains("니다.", regex=False)]
contains_nida.to_csv(OUTPUT_NIDA_FILE, index=False, encoding='utf-8-sig')


# 3. 특수 문자 처리
df['Cleaned Text'] = (
    df['Cleaned Text']
    .str.replace('"', '', regex=False)
    .str.replace("'", '', regex=False)
    .str.replace('#', '', regex=False)
    .str.replace('*', '', regex=False)
    .str.replace(', ', ' ', regex=False)
    .str.replace('-', ' ', regex=False)
    .str.replace('...', '…', regex=False)
    .str.replace('….', '…', regex=False)
    .str.replace(' · ', '·', regex=False)
    .str.replace('· ', '·', regex=False)
    .str.replace(' ·', '·', regex=False)
    .str.replace('  ', ' ', regex=False)
    .str.replace('  ', ' ', regex=False)
    .str.strip()
)

# 4. 후처리 결과 저장
df.rename(columns={'Cleaned Text': 'text'}, inplace=True) # 'Cleaned Text' 열 이름을 'text'로 변경
df.sort_values(by='ID', ascending=True, inplace=True) # 'ID' 열을 기준으로 오름차순 정렬
df.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig') # CSV 파일로 저장

print(f"후처리가 완료되었습니다. 결과는 {OUTPUT_FILE}에 저장되었습니다.")


### Label Cleaning

#### 3. cleaning.py (태원 작성)

In [ ]:
"""
cleanlab을 이용하여 filtering에서 생성한 label_noise.csv를 
label_cleaned.csv로 cleaning하는 코드입니다.
(label_noise 데이터는 postprocess 과정이 필요 없어 진행하지 않습니다.)
"""

INPUT_TEXT_CLEANED = "text_cleaned.csv"
INPUT_LABEL_NOISE = "../1_filtering/5_label_noise.csv"
OUTPUT_DATA = "label_cleaned.csv"

import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import random
import os
import evaluate
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import normalize

# 시드 설정
SEED = 456
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# 토크나이저 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=7)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BASE_DIR = os.getcwd()
OUTPUT_DIR = os.path.join(BASE_DIR, '../outputs')

# 데이터 로드
clean_data = pd.read_csv(os.path.join(INPUT_TEXT_CLEANED))
noise_data = pd.read_csv(os.path.join(INPUT_LABEL_NOISE))

# 데이터셋 클래스 정의
class BERTDataset(Dataset):
    def __init__(self, data, tokenizer):
        input_texts = data['text']
        targets = data['target']
        self.inputs = []; self.labels = []
        for text, label in zip(input_texts, targets):
            tokenized_input = tokenizer(text, padding='max_length', truncation=True, return_tensors='pt')
            self.inputs.append(tokenized_input)
            self.labels.append(torch.tensor(label))

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'].squeeze(0),
            'attention_mask': self.inputs[idx]['attention_mask'].squeeze(0),
            'labels': self.labels[idx].squeeze(0)
        }

    def __len__(self):
        return len(self.labels)

# 데이터 분할 (clean 데이터만 사용)
dataset_train, dataset_valid = train_test_split(clean_data, test_size=0.2, random_state=SEED)

# 데이터셋 및 데이터로더 생성
data_train = BERTDataset(dataset_train, tokenizer)
data_valid = BERTDataset(dataset_valid, tokenizer)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 평가 지표 설정
f1 = evaluate.load('f1')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

# WandB 비활성화
os.environ['WANDB_DISABLED'] = 'true'

# 트레이닝 아규먼트 설정
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    logging_strategy='steps',
    eval_strategy='steps',
    save_strategy='steps',
    logging_steps=100,
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    learning_rate=2e-05,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    seed=SEED
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data_train,
    eval_dataset=data_valid,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 모델 학습
trainer.train()
print("Fine-tuning 완료")

# 임베딩 추출 함수
def get_embeddings(texts, model, tokenizer, device, batch_size=32):
    model.eval()
    model.config.output_hidden_states = True  # hidden states 활성화
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="텍스트 임베딩 중"):
        batch = texts[i:i+batch_size]
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt', max_length=256)
        input_ids = encoded_input['input_ids'].to(device)
        attention_mask = encoded_input['attention_mask'].to(device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            # 마지막 은닉층에서 [CLS] 토큰 벡터 추출
            cls_embeddings = outputs.hidden_states[-1][:, 0, :].cpu().numpy()
        embeddings.append(cls_embeddings)
    # 임베딩 벡터 정규화 (L2 정규화)
    normalized_embeddings = normalize(np.vstack(embeddings), norm='l2')
    return normalized_embeddings

# 클린 데이터와 노이즈 데이터의 임베딩 추출
clean_embeddings = get_embeddings(clean_data['text'].tolist(), model, tokenizer, device)
noise_embeddings = get_embeddings(noise_data['text'].tolist(), model, tokenizer, device)

# RandomForest 분류기 학습 (전체 clean 데이터 사용)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=SEED)
rf_classifier.fit(clean_embeddings, clean_data['target'])

# 노이즈 데이터에 대한 예측
predicted_labels = rf_classifier.predict(noise_embeddings)

# 노이즈 데이터의 라벨 교정
noise_data['corrected_target'] = predicted_labels

# 클린 데이터와 교정된 노이즈 데이터 합치기
combined_data = noise_data[['ID', 'text', 'corrected_target']].rename(columns={'corrected_target': 'target'})

# 결과 저장
combined_data.to_csv(OUTPUT_DATA, index=False, columns=['ID', 'text', 'target'])
print(f"\n교정된 데이터가 {OUTPUT_DATA}에 저장되었습니다.")


### Merge

### 4. merge_cleaned.py

In [ ]:
"""
label_cleaned.csv와 text_cleaned.csv를 합쳐 train_cleaned.csv를 생성하는 코드입니다.
"""

import pandas as pd

df_label = pd.read_csv('label_cleaned.csv')
df_text = pd.read_csv('text_cleaned.csv')

# 병합
df_combined = pd.concat([df_label, df_text], ignore_index=True)

# ID 기준으로 오름차순 정렬
df_sorted = df_combined.sort_values(by='ID')

# CSV로 저장
df_sorted.to_csv('train_cleaned.csv', index=False)